In [1]:
import pandas as pd

In [4]:
all_tables = pd.read_html('https://www.twitch.tv/creatorcamp/en/learn-the-basics/emotes/', header=0)
all_tables[0]

,Emote Code,Emote,Meaning
0,:Kappa:,NaN,Sarcasm or wry humor. Kappa is our signature e...
1,:HeyGuys:,NaN,"A casual greeting. Used when joining chat, or ..."
2,:LUL:,NaN,Laughter. The emote version of Laugh Out Loud.
3,:CoolStoryBob:,NaN,"Our version of “Cool story, bro.” Used sarcast..."
4,:4Head:,NaN,":Laughter, but in a slightly mocking way. Ofte..."
5,:FrankerZ:,NaN,DOG! Used where there’s discussion about a dog...
6,:Jebaited:,NaN,For when someone is “baited or tricked”. Often...
7,:PJSalt:,NaN,When someone gets “salty” at their game or tea...
8,:NotLikeThis:,NaN,"Used to express dismay at an outcome, usually ..."
9,:WutFace:,NaN,"Used to express shock, disgust, or to note a l..."


# Description

Notebook to explore data, recreate bugs, solve bugs

In [1]:
from pillaralgos import algo2
import pillaralgos_dev as p_dev
from pillaralgos_dev import dev_helpers as dev_help
import json

In [2]:
buggy_file = '981828174'
def bug_handler(buggy_file, algo):
    '''
    Simple function to run reported file and files in data folder through a function
    '''
    aws = p_dev.awsBucketAPI()
    aws.save_specific_file(buggy_file)
    data = json.load(open(f'data/{buggy_file}.json'))
    try:
        json_result = algo.run(data)
    except Exception as e:
        print("FAILED")
        print(e)

    import os
    files = os.listdir('data/')
    new_files = [f for f in files if '.json' in f and 'big_data' not in f]

    algoworked = []
    algofailed = []
    results = {}
    for file in new_files:
        try:
            data = json.load(open(f'data/{file}'))
            results[file] = algo.run(data)
            algoworked.append(file)
        except:
            algofailed.append(file)
    return algoworked, algofailed, results

# Unit Test creations

In [1]:
import json
import pandas as pd

In [3]:
data = json.load(open("pypi/prod/test/sample_data/sample_med.json"))

In [6]:
from pillaralgos.helpers import data_handler as dh

In [39]:
from pillaralgos import algo1, algo2, algo3_0, algo3_5

In [37]:
json_re1 = algo1.run(data,min_=0.5, limit=10, sort_by='rel',save_json=False)

In [40]:
json_re2 = algo2.run(data,min_=0.5, limit=10,save_json=False)

/home/jupyter-pomkos/.conda/envs/pillar_env/lib/python3.8/site-packages/pillaralgos/helpers/data_handler.py:278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["hour"] = i
/home/jupyter-pomkos/.conda/envs/pillar_env/lib/python3.8/site-packages/pillaralgos/helpers/data_handler.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["chunk"] = x


In [43]:
json_re3 = algo3_0.run(data,min_=0.5, limit=10, min_words=10,save_json=False)

/home/jupyter-pomkos/.conda/envs/pillar_env/lib/python3.8/site-packages/pillaralgos/helpers/data_handler.py:278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["hour"] = i
/home/jupyter-pomkos/.conda/envs/pillar_env/lib/python3.8/site-packages/pillaralgos/helpers/data_handler.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["chunk"] = x


In [45]:
json_re3 = algo3_5.run(data,min_=0.5, limit=10, goal='num_emo',save_json=False)

/home/jupyter-pomkos/.conda/envs/pillar_env/lib/python3.8/site-packages/pillaralgos/helpers/data_handler.py:278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["hour"] = i
/home/jupyter-pomkos/.conda/envs/pillar_env/lib/python3.8/site-packages/pillaralgos/helpers/data_handler.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["chunk"] = x


In [46]:
json_re3

[{'startTime': 0.0, 'endTime': 119.562},
 {'startTime': 765.512, 'endTime': 882.645},
 {'startTime': 891.677, 'endTime': 1008.99},
 {'startTime': 259.889, 'endTime': 375.413},
 {'startTime': 127.098, 'endTime': 234.249},
 {'startTime': 383.298, 'endTime': 495.599},
 {'startTime': 630.41, 'endTime': 741.233},
 {'startTime': 1018.379, 'endTime': 1131.467},
 {'startTime': 507.003, 'endTime': 626.884},
 {'startTime': 1138.629, 'endTime': 1256.19}]